# core

This is the core and only module of the nbsstorytime library. Outputs the content of all cells (in Jupyter) or messages (in SolveIt) that start with `#| n4l`.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import nbformat

In [ ]:
def is_n4l_cell(cell):
    """Pure function to check if cell has n4l marker"""
    return (cell.cell_type == 'raw' and 
            cell.source.strip().startswith('#| n4l'))

def extract_n4l_content(cell):
    """Pure function to extract content without marker"""
    lines = cell.source.strip().split('\n')
    return '\n'.join(lines[1:]).strip()

def process_notebook(notebook_path):
    """Main processing function using functional composition"""
    with open(notebook_path, 'r') as f:
        notebook = nbformat.read(f, as_version=4)
    
    # Functional pipeline
    return list(map(extract_n4l_content, 
                   filter(is_n4l_cell, notebook.cells)))


In [ ]:
#| export
import inspect

In [ ]:
#| export
def find_dialog_name():
    for frame_info in inspect.stack():
        if '__dialog_name' in frame_info.frame.f_locals:
            return frame_info.frame.f_locals['__dialog_name']
    return None
    

In [ ]:
from ipykernel import get_connection_info

In [ ]:
# Get full connection details
conn_info = None
try:
    conn_info = get_connection_info()
except RuntimeError as e:
    print("not running in notebook", e)
conn_info

In [ ]:
import json

In [ ]:
#| export
import json
from ipykernel import get_connection_info
from fastcore.basics import Path
from fastcore.basics import patch
import nbformat

In [ ]:
#| export
N4L_SOLVEIT_ROOT = Path('/app/data')

In [ ]:
#| export
def find_current_notebook():
    # Try SolveIt first (via __dialog_name in call stack)
    dname = find_dialog_name()        
    if(dname):
        dname = dname + ".ipynb"
        return N4L_SOLVEIT_ROOT / dname 
    else:    
        # Fall back to Jupyter
        try:
            conn_info = get_connection_info()
            if conn_info:
                conn_data = json.loads(conn_info)
                jupyter_session = conn_data.get('jupyter_session')
                if jupyter_session:
                    return Path(jupyter_session)
        except RuntimeError:
            pass
    
    return None

In [ ]:
nb = None
try:
    nb = find_current_notebook()
except: pass
nb

In [ ]:
#| export
@patch
def export_n4l_content(self:Path):
    """Add n4l export capability to any Path object"""
    # Load the notebook
    notebook = nbformat.read(self.open(), as_version=4)
    
    # Find n4l cells (your existing logic)
    n4l_contents = []
    for cell in notebook.cells:
        if (cell.cell_type == 'raw' and 
            cell.source.strip().startswith('#| n4l')):
            # Remove the marker line
            lines = cell.source.strip().split('\n')
            content = '\n'.join(lines[1:]).strip()
            n4l_contents.append(content)
    
    # Create output file
    output_path = self.with_suffix('.n4l')
    combined_content = '\n\n'.join(n4l_contents)
    output_path.write_text(combined_content)
    
    return output_path

In [ ]:
#| export
def nbsstorytime_export():
    current_path = find_current_notebook()
    if(current_path): 
        current_path.export_n4l_content()
    else:
        print("no current notebook")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()